In [1]:
!pip install geopandas

     |████████████████████████████████| 921kB 3.4MB/s 
     |████████████████████████████████| 10.1MB 34.0MB/s 
     |████████████████████████████████| 11.8MB 29.7MB/s 


In [0]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import branca.colormap as cm
import datetime as dt

In [0]:
data = pd.read_csv("train.csv")

In [0]:
data["Dates"] = pd.to_datetime(data["Dates"])
data["Year"] = data["Dates"].dt.year
data["Month"] = data["Dates"].dt.month
data["Day"] = data["Dates"].dt.day

In [5]:
data.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,Year,Month,Day
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,2015,5,13
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,2015,5,13
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,2015,5,13
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,2015,5,13


In [63]:
map_test = data.copy()
map_test = map_test[map_test["Year"] == 2015]
map_test = map_test[map_test["Month"] == 1]
map_test = map_test[map_test["Day"] == 5]
map_test = map_test.drop(columns=["Descript", "DayOfWeek", "Resolution", "Address", "X", "Y", "Dates", "Year","Month","Day"])
map_test = map_test.groupby(by=["PdDistrict"]).agg("count")
map_test["Crimes"] = map_test["Category"]
map_test = map_test.drop(columns=["Category"])
map_test.reset_index(inplace=True)
map_test

,PdDistrict,Crimes
0,BAYVIEW,48
1,CENTRAL,73
2,INGLESIDE,41
3,MISSION,48
4,NORTHERN,51
5,PARK,23
6,RICHMOND,15
7,SOUTHERN,84
8,TARAVAL,38
9,TENDERLOIN,26


In [0]:
sfpd = gpd.read_file("current_police_districts.geojson")

In [64]:
sfpd=sfpd[["district","geometry"]]
sfpd.columns=["PdDistrict", "geometry"]
sfpd.head()

,PdDistrict,geometry
0,SOUTHERN,"MULTIPOLYGON (((-122.39186 37.79425, -122.3917..."
1,BAYVIEW,"MULTIPOLYGON (((-122.38098 37.76480, -122.3810..."
2,MISSION,"MULTIPOLYGON (((-122.40954 37.76932, -122.4086..."
3,NORTHERN,"MULTIPOLYGON (((-122.43379 37.80793, -122.4337..."
4,TENDERLOIN,"MULTIPOLYGON (((-122.40217 37.78626, -122.4171..."


In [0]:
map_test_all = sfpd.merge(map_test,on="PdDistrict")

In [67]:
map_test_all

,PdDistrict,geometry,Crimes
0,SOUTHERN,"MULTIPOLYGON (((-122.39186 37.79425, -122.3917...",84
1,BAYVIEW,"MULTIPOLYGON (((-122.38098 37.76480, -122.3810...",48
2,MISSION,"MULTIPOLYGON (((-122.40954 37.76932, -122.4086...",48
3,NORTHERN,"MULTIPOLYGON (((-122.43379 37.80793, -122.4337...",51
4,TENDERLOIN,"MULTIPOLYGON (((-122.40217 37.78626, -122.4171...",26
5,CENTRAL,"MULTIPOLYGON (((-122.42612 37.80684, -122.4261...",73
6,PARK,"MULTIPOLYGON (((-122.43956 37.78314, -122.4383...",23
7,RICHMOND,"MULTIPOLYGON (((-122.44127 37.79149, -122.4406...",15
8,INGLESIDE,"MULTIPOLYGON (((-122.40450 37.74858, -122.4040...",41
9,TARAVAL,"MULTIPOLYGON (((-122.49842 37.70810, -122.4984...",38


In [90]:
colormap = cm.linear.YlGnBu_09.to_step(data=map_test_all["Crimes"], method="quant", quantiles=[0,0.25,0.5,0.75, 1])
colormap

In [91]:
colormap.caption = "# of Crimes"
mymap = folium.Map(location=center, zoom_start=11,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)
colormap.caption = "% of foreign resident population"
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x["properties"]["Crimes"]), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
info=folium.features.GeoJson(
        map_test_all,
        style_function=style_function,
        control=False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(fields=["PdDistrict","Crimes"],
            aliases=["Police Department District: ","# of Crimes"],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
    )
colormap.add_to(mymap)
mymap.add_child(info)
mymap

/usr/local/lib/python3.6/dist-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [0]:
sfmap.save("test.html")